<a href="https://colab.research.google.com/github/ibrahimasifali94/sql_ab_test_metrics/blob/main/GH_sql_a_b_test_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install duckdb pandas numpy

In [2]:
import numpy as np, pandas as pd
rng = np.random.default_rng(7)

N = 5000
user_ids = np.arange(1, N+1)

groups = rng.choice(["A","B"], size=N)
p_click_A, lift = 0.05, 0.15
p_click_B = p_click_A*(1+lift)

clicks = np.where(groups=="A",
                  rng.binomial(1, p_click_A, N),
                  rng.binomial(1, p_click_B, N))
ret = rng.binomial(1, np.where(clicks==1, 0.40, 0.10), N)

users = pd.DataFrame({"user_id": user_ids})
assignments = pd.DataFrame({"user_id": user_ids, "variant": groups})
events = []
for u,g,c,r in zip(user_ids, groups, clicks, ret):
    if c: events.append([u,"click",0])
    if r: events.append([u,"login",1])
events = pd.DataFrame(events, columns=["user_id","event","day_offset"])

users.to_csv("users.csv", index=False)
assignments.to_csv("assignments.csv", index=False)
events.to_csv("events.csv", index=False)

users.head(), assignments.head(), events.head()


(   user_id
 0        1
 1        2
 2        3
 3        4
 4        5,
    user_id variant
 0        1       B
 1        2       B
 2        3       B
 3        4       B
 4        5       B,
    user_id  event  day_offset
 0        6  login           1
 1       10  click           0
 2       12  login           1
 3       13  login           1
 4       14  login           1)

In [4]:
import scipy.stats as st

df = con.execute("""
WITH exposure AS (
  SELECT a.user_id, a.variant FROM assignments a
),
clicks AS (
  SELECT e.user_id, 1 AS clicked
  FROM events e
  WHERE e.event='click' AND e.day_offset=0
  GROUP BY 1
),
d1 AS (
  SELECT e.user_id, 1 AS d1_retained
  FROM events e
  WHERE e.event='login' AND e.day_offset=1
  GROUP BY 1
),
user_metrics AS (
  SELECT ex.user_id, ex.variant,
         COALESCE(c.clicked,0) AS clicked,
         COALESCE(d.d1_retained,0) AS d1_retained
  FROM exposure ex
  LEFT JOIN clicks c ON ex.user_id=c.user_id
  LEFT JOIN d1 d ON ex.user_id=d.user_id
),
agg AS (
  SELECT variant,
         COUNT(*)::DOUBLE AS n,
         SUM(clicked)::DOUBLE AS clicks,
         AVG(clicked)::DOUBLE AS ctr,
         AVG(d1_retained)::DOUBLE AS d1
  FROM user_metrics GROUP BY 1
)
SELECT * FROM agg;
""").df()

df


,variant,n,clicks,ctr,d1
0,A,2497.0,116.0,0.046456,0.111334
1,B,2503.0,149.0,0.059529,0.116260


In [5]:
# split into groups
a = df[df.variant=="A"].iloc[0]
b = df[df.variant=="B"].iloc[0]

p_a, n_a = a.ctr, a.n
p_b, n_b = b.ctr, b.n

diff = p_b - p_a
se = ((p_a*(1-p_a)/n_a) + (p_b*(1-p_b)/n_b))**0.5
z = diff / se
p_value = 2*(1 - st.norm.cdf(abs(z)))

print(f"CTR A: {p_a:.4f}, CTR B: {p_b:.4f}")
print(f"Absolute lift: {diff:.4%}, Relative lift: {p_b/p_a - 1:.2%}")
print(f"z = {z:.2f}, p-value = {p_value:.4f}")


CTR A: 0.0465, CTR B: 0.0595
Absolute lift: 1.3073%, Relative lift: 28.14%
z = 2.06, p-value = 0.0390


## Results
- CTR A: 4.65%
- CTR B: 5.95%
- Absolute Lift: 1.3073%
- Relative lift: +28.14%
- p-value: 0.039 (statistically significant at 95%)
- D1 retention is unchanged.
